# Consommation électricité & gaz 20K - 50K


Ce notebook sert à grouper les données ouvertes nécesaires à la dataviz et à le préparer à l'importation en base de données.

## Fichiers csv vs base de données

La première approche a consisté à grouper dans un fichier csv toutes les données concernant les communes et celles fournies par ORE. En comptant seulement les collectivités entre 20K et 50K habitants, **le fichier csv faisait plusieurs Mo**.

Cela semble trop pour un usage avec un smartphone.

Ce notebook tente une autre approche : créer une base de données à laquelle on sollicite seulement les données concernant la collectivité qui nous intéresse. Ces requêtes font **entre 1 ko et 6 ko**.

Cela rend un peu plus longue la préparation des données mais améliore grandement l'expérience des utilisateurs en conditions de mobilité.

## Structure 

** organismes **
id, libelle, superficie, habitants, epci, code_epci, dept, code_dept, region, code_region, insee, code_postal, forme, geolocalisation 


** donnees **


In [1]:
import pandas as pd 
import glob
#import numpy as np

In [2]:
import pymysql.cursors

connection = pymysql.connect( host='localhost',
                              user='root',
                              passwd='root',
                              db='energie',
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

# Table consommation électricité & gaz

On va importer les données ORE électricité, et on va les concatener avec celles du gaz.

Avant de tout mettre en BDD, on va filtrer les données des communes 20K-50K, pour cela on va importer les données Insee des communes

In [3]:
# Electricité
edf = pd.read_csv('data/ore/conso-elec-annuelle-par-secteur-dactivite-agregee-commune.csv',
                 sep=';', thousands=' ', dtype = str,
                 usecols=['Année', 'Filière', 'Code Commune', 
                           'Consommation Agriculture (MWh)', 
                           'Consommation Industrie (MWh)',  'Consommation Tertiaire  (MWh)', 
                           'Consommation Résidentiel  (MWh)', 'Consommation Secteur Inconnu (MWh)', 
                           'Consommation totale', 
                          'Nombre de points Agriculture', 'Nombre de points Industrie',
                           'Nombre de points Tertiaire', 'Nombre de points Résidentiel',
                           'Nombre de points Secteur Inconnu', 
                           'Nombre de mailles secretisées (résidentiel)',
                           'Nombre de mailles secretisées (secteur inconnu)'])
edf = edf.rename(columns={'Année': 'annee', 'Filière': 'filiere', 'Code Commune': 'insee',
                          'Consommation Agriculture (MWh)': 'conso_agriculture', 
                           'Consommation Industrie (MWh)': 'conso_industrie',  
                          'Consommation Tertiaire  (MWh)': 'conso_tertiaire', 
                           'Consommation Résidentiel  (MWh)': 'conso_residentiel', 
                          'Consommation Secteur Inconnu (MWh)': 'conso_inconu', 
                           'Consommation totale': 'conso_totale', 
                          'Nombre de points Agriculture': 'points_agriculture', 
                          'Nombre de points Industrie': 'points_industrie',
                           'Nombre de points Tertiaire': 'points_tertiaire', 
                          'Nombre de points Résidentiel': 'points_residentiel',
                           'Nombre de points Secteur Inconnu': 'points_inconu', 
                           'Nombre de mailles secretisées (résidentiel)': 'mailles_residentiel',
                           'Nombre de mailles secretisées (secteur inconnu)': 'mailles_inconu'})

edf = edf.drop_duplicates(subset=None, keep='first')

edf.shape

(209549, 16)

In [4]:
gdf = pd.read_csv('data/ore/conso-gaz-annuelle-par-secteur-dactivite-agregee-commune.csv',
                 sep=';', thousands=' ', dtype = str,
                 usecols=['Année', 'Filière', 'Code Commune', 
                           'Consommation Agriculture (MWh)', 
                           'Consommation Industrie (MWh)',  'Consommation Tertiaire  (MWh)', 
                           'Consommation Résidentiel  (MWh)', 'Consommation Secteur Inconnu (MWh)', 
                           'Consommation totale', 
                          'Nombre de points Agriculture', 'Nombre de points Industrie',
                           'Nombre de points Tertiaire', 'Nombre de points Résidentiel',
                           'Nombre de points Secteur Inconnu', 
                           'Nombre de mailles secretisées (résidentiel)',
                           'Nombre de mailles secretisées (secteur inconnu)'])
gdf = gdf.rename(columns={'Année': 'annee', 'Filière': 'filiere', 'Code Commune': 'insee',
                          'Consommation Agriculture (MWh)': 'conso_agriculture', 
                           'Consommation Industrie (MWh)': 'conso_industrie',  
                          'Consommation Tertiaire  (MWh)': 'conso_tertiaire', 
                           'Consommation Résidentiel  (MWh)': 'conso_residentiel', 
                          'Consommation Secteur Inconnu (MWh)': 'conso_inconu', 
                           'Consommation totale': 'conso_totale', 
                          'Nombre de points Agriculture': 'points_agriculture', 
                          'Nombre de points Industrie': 'points_industrie',
                           'Nombre de points Tertiaire': 'points_tertiaire', 
                          'Nombre de points Résidentiel': 'points_residentiel',
                           'Nombre de points Secteur Inconnu': 'points_inconu', 
                           'Nombre de mailles secretisées (résidentiel)': 'mailles_residentiel',
                           'Nombre de mailles secretisées (secteur inconnu)': 'mailles_inconu'})
gdf.head(3)

,annee,filiere,conso_agriculture,points_agriculture,conso_industrie,points_industrie,conso_tertiaire,points_tertiaire,conso_residentiel,points_residentiel,conso_inconu,points_inconu,insee,mailles_residentiel,mailles_inconu,conso_totale
0,2016,Gaz,0.0,0.0,0.0,0.0,0.0,0.0,3671.0,217.0,0.0,0.0,22324,0,0,3671.0
1,2016,Gaz,0.0,0.0,0.0,0.0,0.0,0.0,2624.0,186.0,0.0,0.0,22307,0,0,2624.0
2,2016,Gaz,0.0,0.0,9775.0,3.0,144.0,1.0,1031.0,42.0,0.0,0.0,22277,0,0,10950.0


In [5]:
# concatenate 
egdf = pd.concat([edf, gdf])
egdf = egdf.fillna('')
egdf = egdf.drop_duplicates(subset=None, keep='first')

egdf.shape

(267882, 16)

In [6]:
egdf.loc[egdf['insee'] == '01001']

,annee,filiere,conso_agriculture,points_agriculture,conso_industrie,points_industrie,conso_tertiaire,points_tertiaire,conso_residentiel,points_residentiel,conso_inconu,points_inconu,insee,mailles_residentiel,mailles_inconu,conso_totale
19591,2016,Electricité,0.0,0.0,53.06,1.0,244.4,2.0,3015.79,359.0,0.0,0.0,01001,0,0,3313.25
60329,2011,Electricité,0.0,0.0,62.69,1.0,239.06,2.0,2711.47,346.0,0.0,0.0,01001,0,0,3013.22
109157,2013,Electricité,0.0,0.0,53.07,1.0,256.0,2.0,3025.92,346.0,0.0,0.0,01001,0,0,3334.9900000000002
140208,2014,Electricité,0.0,0.0,52.96,1.0,249.9,2.0,2718.12,353.0,0.0,0.0,01001,0,0,3020.98
157484,2015,Electricité,0.0,0.0,53.3,1.0,239.43,2.0,2858.96,355.0,0.0,0.0,01001,0,0,3151.69
160503,2012,Electricité,0.0,0.0,57.23,1.0,236.1,2.0,2919.76,345.0,0.0,0.0,01001,0,0,3213.09


In [4]:
# garder que 20k - 50k
# données population des communes
# source : https://www.insee.fr/fr/statistiques/2521169#consulter

pop = pd.read_csv('data/insee/base_cc.csv', encoding='utf-8', sep=',', 
                  low_memory=False, keep_default_na = False )
                 #dtype={'CODGEO': str, 'P15_POP': np.int64})
pop = pop.rename(columns={'CODGEO': 'insee' , 'P15_POP': 'population'})

pop = pop.loc[pop['population'] > 19999]
pop = pop.loc[pop['population'] < 50001]
pop = pop.fillna('')
pop = pop.drop_duplicates(subset=None, keep='first')

pop.shape

(353, 19)

In [31]:
tout = pd.merge(pop, egdf, how='right')
tout = tout.dropna(how='any')
tout = tout.drop_duplicates(subset=None, keep='first')
tout.shape

(3805, 34)

## to sql

In [26]:
# Insertion BDD
# Il a été nécessaire de changer les NaN en ''

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO `consomation_20_50` "
           "( `insee`, `annee`, `filiere`, `conso_totale`, `conso_agriculture`," 
           "  `points_agriculture`, `conso_industrie`, `points_industrie`, `conso_tertiaire`, `points_tertiaire`, "
           " `conso_residentiel`, `points_residentiel`,  `conso_inconu`, `points_inconu`, "
           " `mailles_residentiel`, `mailles_inconu`)"
           "VALUES ("
           " %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,   %s, %s, %s, %s, %s, %s)"
          )

    for i, r in tout.iterrows():
        cursor.execute(sql, 
            (r['insee'], r['annee'], r['filiere'], r['conso_totale'], r['conso_agriculture'], 
            r['points_agriculture'], r['conso_industrie'], r['points_industrie'], r['conso_tertiaire'], r['points_tertiaire'], 
            r['conso_residentiel'], r['points_residentiel'], r['conso_inconu'], r['points_inconu'], 
            r['mailles_residentiel'], r['mailles_inconu'] )
        )

connection.commit() 

## Chaud froid

In [6]:
# FILIERE : il y a 'C' (chaleur) et 'F' (Froid)
# TYPE : Il n'y a que des communes


e = 'iso-8859-15'
s = ';'

cf = pd.concat([pd.read_csv(f, dtype=str, encoding=e, sep=s) for f in glob.glob('data/chaleur-froid/*.csv')])

cf.head()

,OPERATEUR,ANNEE,FILIERE,TYPE,CODE,PDL,PUISSANCE,PRODTOT,PCTCOG,DPE,CONSOR,CONSOT,CONSOI,CONSOA,CONSONA
0,Quartier de La Reyssouze,2015,C,Commune,01053,74,"39,9",24839,"25,62408006","0,142","26654,1","5850,9",0,0,0
1,Réseau de chaleur d'Hauteville Lompnes,2015,C,Commune,01185,20,7,7575,0,"0,046",-99,"3637,6051",0,0,0
2,Réseau de saint-denis-les-bourg,2015,C,Commune,01344,10,4,1323,0,NaN,0,1052,0,0,0
3,Belena,2015,C,Commune,01234,25,12,12229,0,NaN,-99,6333,0,0,0
4,Réseau de chaleur de TREFFORT,2015,C,Commune,01426,19,"0,5","628,43",0,"0,027",0,0,0,0,"563,55"


In [7]:
cf.shape

(4351, 15)

In [8]:
cf = cf.fillna('')
cf.shape

(4351, 15)

In [9]:
cf = cf.dropna(how='any')
cf.shape

(4351, 15)

In [45]:
cf = cf.dropna(how='any')
cf = cf.replace(np.nan, 0)

In [10]:
toutcf = pd.merge(cf, pop, left_on='CODE', right_on='insee', how='inner')
toutcf.head(3)

,OPERATEUR,ANNEE,FILIERE,TYPE,CODE,PDL,PUISSANCE,PRODTOT,PCTCOG,DPE,...,P15_EMPLT_SAL,ETTOT15,ETAZ15,ETBE15,ETFZ15,ETGU15,ETGZ15,ETOQ15,ETTEF115,ETTEFP1015
0,CEF - MCO,2015,C,Commune,11069,2,"7,5","9204,422",0,,...,25509,5664,82,245,528,3857,1273,952,1658,470
1,CEF - MCO,2014,C,Commune,11069,2,"10,69","6823,55",0,"0,250268806",...,25509,5664,82,245,528,3857,1273,952,1658,470
2,Réseau Sarrus,2015,C,Commune,12202,5,"3,28","2182,74",0,"0,071",...,18785,3286,19,139,258,2265,551,605,993,335


In [11]:
toutcf.shape

(1188, 34)

In [12]:
toutcf.to_csv('toutcf.csv')

In [13]:
toutcf

,OPERATEUR,ANNEE,FILIERE,TYPE,CODE,PDL,PUISSANCE,PRODTOT,PCTCOG,DPE,...,P15_EMPLT_SAL,ETTOT15,ETAZ15,ETBE15,ETFZ15,ETGU15,ETGZ15,ETOQ15,ETTEF115,ETTEFP1015
0,CEF - MCO,2015,C,Commune,11069,2,"7,5","9204,422",0,,...,25509,5664,82,245,528,3857,1273,952,1658,470
1,CEF - MCO,2014,C,Commune,11069,2,"10,69","6823,55",0,"0,250268806",...,25509,5664,82,245,528,3857,1273,952,1658,470
2,Réseau Sarrus,2015,C,Commune,12202,5,"3,28","2182,74",0,"0,071",...,18785,3286,19,139,258,2265,551,605,993,335
3,Réseau Sarrus,2013,C,Commune,12202,5,"3,31","2674,2036",0,"0,247541355",...,18785,3286,19,139,258,2265,551,605,993,335
4,Réseau Sarrus,2014,C,Commune,12202,5,"3,28","2035,2928",0,,...,18785,3286,19,139,258,2265,551,605,993,335
5,Zac des Canourgues,2015,C,Commune,13103,28,"33,7",38400,"55,5421343","0,03",...,16778,4526,108,209,485,2975,917,749,1130,253
6,ZAC des Canourgues,2013,C,Commune,13103,27,"27,4","38292,78","30,3268658","0,118614887",...,16778,4526,108,209,485,2975,917,749,1130,253
7,Zac des Canourgues,2014,C,Commune,13103,27,"33,8","34656,86","57,21522377","0,074598895",...,16778,4526,108,209,485,2975,917,749,1130,253
8,Zac des Canourgues,2011,C,Commune,13103,22,"27,4","34169,1712","54,62526407","0,216678376",...,16778,4526,108,209,485,2975,917,749,1130,253
9,Zac des Canourgues,2012,C,Commune,13103,27,"27,4","39730,5091","36,951452","0,214",...,16778,4526,108,209,485,2975,917,749,1130,253


In [14]:
# Insertion BDD
# ???? Impossible de remplir les NaN de la colonne DPE ????

with connection.cursor() as cursor:
    
    sql = ("INSERT INTO chaleur_froid_20_50 "
           "( insee, operateur, annee, filiere, pdl, " 
           "  puissance, prod_tot, pct_cog, dpe, conso_agriculture,  "
           "  conso_inconu, conso_industrie ,conso_residentiel, conso_tertiaire "
           " )"
           "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,   %s, %s, %s, %s)"
          )

    for i, r in toutcf.iterrows():
        cursor.execute(sql, 
            (r['CODE'], 
             r['OPERATEUR'], 
             r['ANNEE'], 
             r['FILIERE'], 
             r['PDL'], 
             r['PUISSANCE'], 
             r['PRODTOT'], 
             r['PCTCOG'],  
             #r['DPE'], 
             '',
             r['CONSOA'],  
             r['CONSONA'], 
             r['CONSOI'], 
             r['CONSOR'], 
             r['CONSOT']  
            )
        )

connection.commit() 